# Data Preprocessing for Spotify and YouTube Music Analysis

This notebook performs preprocessing on two datasets:
1. Spotify-YouTube Dataset
2. Most Streamed Spotify Songs 2024

The preprocessing steps include:
- Loading and examining datasets
- Cleaning and standardizing data
- Handling duplicates
- Combining datasets

## Import Required Libraries

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

## Define Helper Functions

In [2]:
def normalize_column_names(df):
    """
    Normalize column names to snake_case format
    """
    return df.rename(columns=lambda x: x.lower().replace(' ', '_'))

def clean_text_data(text):
    """
    Clean text data by removing special characters and standardizing format
    """
    if pd.isna(text):
        return text
    # Remove special characters and extra spaces
    cleaned = str(text).strip().lower()
    # Replace multiple spaces with single space
    cleaned = ' '.join(cleaned.split())
    return cleaned

## Load and Examine Datasets

In [3]:
# Load datasets with encoding handling
spotify_youtube = pd.read_csv('data/raw/Spotify_Youtube.csv', encoding='latin1')
spotify_2024 = pd.read_csv('data/raw/Most Streamed Spotify Songs 2024.csv', encoding='latin1')

print("Spotify YouTube Dataset Info:")
print(spotify_youtube.info())
print("\nMissing values in Spotify YouTube Dataset:")
print(spotify_youtube.isnull().sum())

print("\nSpotify 2024 Dataset Info:")
print(spotify_2024.info())
print("\nMissing values in Spotify 2024 Dataset:")
print(spotify_2024.isnull().sum())

Spotify YouTube Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20718 entries, 0 to 20717
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Id                20718 non-null  int64  
 1   Artist            20718 non-null  object 
 2   Url_spotify       20718 non-null  object 
 3   Track             20718 non-null  object 
 4   Album             20718 non-null  object 
 5   Album_type        20718 non-null  object 
 6   Uri               20718 non-null  object 
 7   Danceability      20716 non-null  float64
 8   Energy            20716 non-null  float64
 9   Key               20716 non-null  float64
 10  Loudness          20716 non-null  float64
 11  Speechiness       20716 non-null  float64
 12  Acousticness      20716 non-null  float64
 13  Instrumentalness  20716 non-null  float64
 14  Liveness          20716 non-null  float64
 15  Valence           20716 non-null  float64
 16  Tempo     

## Preprocess Spotify YouTube Dataset

In [4]:
def preprocess_spotify_youtube(df):
    """
    Preprocess Spotify YouTube dataset
    """
    # Normalize column names
    df = normalize_column_names(df)
    
    # Clean text columns
    text_columns = ['track', 'artist', 'album']
    for col in text_columns:
        df[col] = df[col].apply(clean_text_data)
    
    # Convert duration from ms to seconds
    df['duration_sec'] = df['duration_ms'] / 1000
    
    # Convert stream count to numeric, handling any non-numeric values
    df['stream'] = pd.to_numeric(df['stream'], errors='coerce')
    
    # Select relevant columns
    relevant_columns = [
        'track', 'artist', 'album', 'danceability', 'energy', 'key',
        'loudness', 'speechiness', 'acousticness', 'instrumentalness',
        'liveness', 'valence', 'tempo', 'duration_sec', 'stream',
        'views', 'likes', 'comments'
    ]
    
    return df[relevant_columns]

spotify_youtube_clean = preprocess_spotify_youtube(spotify_youtube)
print("Preprocessed Spotify YouTube Dataset Info:")
print(spotify_youtube_clean.info())

Preprocessed Spotify YouTube Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20718 entries, 0 to 20717
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   track             20718 non-null  object 
 1   artist            20718 non-null  object 
 2   album             20718 non-null  object 
 3   danceability      20716 non-null  float64
 4   energy            20716 non-null  float64
 5   key               20716 non-null  float64
 6   loudness          20716 non-null  float64
 7   speechiness       20716 non-null  float64
 8   acousticness      20716 non-null  float64
 9   instrumentalness  20716 non-null  float64
 10  liveness          20716 non-null  float64
 11  valence           20716 non-null  float64
 12  tempo             20716 non-null  float64
 13  duration_sec      20716 non-null  float64
 14  stream            20142 non-null  float64
 15  views             20248 non-null  float64
 1

## Preprocess Spotify 2024 Dataset

In [5]:
def preprocess_spotify_2024(df):
    """
    Preprocess Spotify 2024 dataset
    """
    # First normalize all column names to lowercase with underscores
    df = normalize_column_names(df)
    
    # Clean text columns
    text_columns = ['track', 'artist', 'album_name']
    for col in text_columns:
        df[col] = df[col].apply(clean_text_data)
    
    # Convert release_date to datetime
    df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
    
    # Convert numeric columns to appropriate types
    numeric_columns = ['spotify_streams', 'youtube_views', 'youtube_likes', 
                      'tiktok_posts', 'tiktok_likes', 'tiktok_views']
    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col].str.replace(',', ''), errors='coerce')
    
    # Rename columns for consistency with the other dataset
    df = df.rename(columns={
        'album_name': 'album',
        'spotify_streams': 'stream',
        'youtube_views': 'views',
        'youtube_likes': 'likes'
    })
    
    # Select relevant columns
    relevant_columns = [
        'track', 'artist', 'album', 'release_date', 'stream',
        'views', 'likes', 'tiktok_posts', 'tiktok_likes', 'tiktok_views'
    ]
    
    return df[relevant_columns]

spotify_2024_clean = preprocess_spotify_2024(spotify_2024)
print("Preprocessed Spotify 2024 Dataset Info:")
print(spotify_2024_clean.info())

Preprocessed Spotify 2024 Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4600 entries, 0 to 4599
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   track         4600 non-null   object        
 1   artist        4595 non-null   object        
 2   album         4600 non-null   object        
 3   release_date  4600 non-null   datetime64[ns]
 4   stream        4487 non-null   float64       
 5   views         4292 non-null   float64       
 6   likes         4285 non-null   float64       
 7   tiktok_posts  3427 non-null   float64       
 8   tiktok_likes  3620 non-null   float64       
 9   tiktok_views  3619 non-null   float64       
dtypes: datetime64[ns](1), float64(6), object(3)
memory usage: 359.5+ KB
None


## Handle Duplicates

In [6]:
def identify_duplicates(df1, df2):
    """
    Identify duplicate songs between datasets based on track and artist
    """
    # Create a composite key for comparison
    df1['composite_key'] = df1['track'] + '|' + df1['artist']
    df2['composite_key'] = df2['track'] + '|' + df2['artist']
    
    # Find duplicates
    duplicates = set(df1['composite_key']).intersection(set(df2['composite_key']))
    
    print(f"\nNumber of duplicate songs found: {len(duplicates)}")
    if len(duplicates) > 0:
        print("\nSample of duplicate songs:")
        for key in list(duplicates)[:5]:
            print(f"- {key.replace('|', ' by ')}")
    
    return duplicates

duplicates = identify_duplicates(spotify_youtube_clean, spotify_2024_clean)


Number of duplicate songs found: 1303

Sample of duplicate songs:
- flowers by miley cyrus
- old town road - remix by lil nas x
- solo me dejaste by grupo marca registrada
- el rescate by grupo marca registrada
- bones by imagine dragons


## Combine Datasets

In [7]:
# Remove composite key used for duplicate detection
spotify_youtube_clean = spotify_youtube_clean.drop('composite_key', axis=1)
spotify_2024_clean = spotify_2024_clean.drop('composite_key', axis=1)

# For duplicate songs, prefer the newer 2024 dataset
spotify_youtube_clean = spotify_youtube_clean[
    ~(spotify_youtube_clean['track'] + '|' + spotify_youtube_clean['artist']).isin(duplicates)
]

# Identify common and unique columns
common_columns = list(set(spotify_youtube_clean.columns) & set(spotify_2024_clean.columns))
unique_to_youtube = list(set(spotify_youtube_clean.columns) - set(spotify_2024_clean.columns))
unique_to_2024 = list(set(spotify_2024_clean.columns) - set(spotify_youtube_clean.columns))

# Create empty columns in each dataset for the unique columns from the other dataset
for col in unique_to_youtube:
    spotify_2024_clean[col] = np.nan
for col in unique_to_2024:
    spotify_youtube_clean[col] = np.nan

# Combine the datasets
combined_df = pd.concat([spotify_youtube_clean, spotify_2024_clean], ignore_index=True)

print("\nCombined Dataset Info:")
print(combined_df.info())
print("\nSample of combined dataset:")
print(combined_df.head())


Combined Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24012 entries, 0 to 24011
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   track             24012 non-null  object        
 1   artist            24007 non-null  object        
 2   album             24012 non-null  object        
 3   danceability      19410 non-null  float64       
 4   energy            19410 non-null  float64       
 5   key               19410 non-null  float64       
 6   loudness          19410 non-null  float64       
 7   speechiness       19410 non-null  float64       
 8   acousticness      19410 non-null  float64       
 9   instrumentalness  19410 non-null  float64       
 10  liveness          19410 non-null  float64       
 11  valence           19410 non-null  float64       
 12  tempo             19410 non-null  float64       
 13  duration_sec      19410 non-null  float64       
 14

/tmp/ipykernel_25961/548717274.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([spotify_youtube_clean, spotify_2024_clean], ignore_index=True)


## Save Preprocessed Dataset

In [8]:
# Save the combined dataset
combined_df.to_csv('data/preprocessed/combined_dataset.csv', index=False)
print("\nData preprocessing completed successfully!")
print(f"Combined dataset shape: {combined_df.shape}")


Data preprocessing completed successfully!
Combined dataset shape: (24012, 22)
